In [3]:
# run this to read in the rent registry

import pandas as pd
import numpy as np
import usaddress

def cleanAddressLine1(a):
    if pd.isna(a):
        return ''
    if isinstance(a,str) and '10144 CULVER CITY' in a:
        return '10144 CULVER BLVD'
    if isinstance(a,str) and '3959 AND 3961 BENTLEY AVE' in a:
        return '3959-3961 BENTLEY AVE'
    if isinstance(a,str) and '4222 & 4224 JASMINE AVE' in a:
        return '4222-4224 JASMINE AVE'
    if isinstance(a,str) and '4070 LAFAYETTE PL # House B' in a:
        return '4070 LAFAYETTE PL House B'
    if isinstance(a,str) and '4350-68 OVERLOAD AVE' in a:
        return '4350-68 OVERLAND AVE'
    if isinstance(a,str) and 'Do not own a rental property in Culver City' in a:
        return ''
    tagged_address, address_type= usaddress.tag(str(a))
    if 'StreetNamePostType' not in tagged_address.keys():
        return ' '.join([tagged_address['AddressNumber'], tagged_address['StreetName']])
    else:
        return ' '.join([tagged_address['AddressNumber'], tagged_address['StreetName'], tagged_address['StreetNamePostType']])

fn = 'Rent Registry_PRA 22-496.xlsx'
#fn = 'test.xlsx'
ccrr = pd.read_excel(fn)
#ccrr['AddressLine1'] = ccrr['AddressLine1'].str.replace('10144 CULVER CITY','10144 CULVER BLVD')
ccrr['AddressLine1'] = ccrr['AddressLine1'].apply(cleanAddressLine1)
ccrr = ccrr.sort_values(by='PastRentReportDate',ascending=False).reset_index()


    

In [74]:
from geocodio import GeocodioClient
import time

key = 'd8f821033f22f02a2d80862a2286d6d2803dd2a'
client = GeocodioClient(key)

# run this to geocode all the addresses in the rent registry
# make sure you use the exact same usaddress format as below in the "address" variable
existing_geocode = pd.read_csv('cc_geocoded.csv')
addresses = ccrr.drop_duplicates(['AddressLine1'])['AddressLine1']
#addresses = addresses.apply(cleanAddressLine1).drop_duplicates()
headers = ['address','lat','lon']
geocoded = existing_geocode.copy()
l = 0
addresses_to_geocode = []
for address in addresses:
    if address == '':
        continue
    elif address in existing_geocode['address'].values:
        continue
    addresses_to_geocode.append(address)

    l += 1
    if l > 10000:
        break

if len(addresses_to_geocode) > 0:
    geocoded_location = client.batch_geocode([s + ', CULVER CITY, CA' for s in addresses_to_geocode])
    coords = geocoded_location.coords
    #coords = []
    #for address in addresses_to_geocode:
    #    lat = l*20
    #    lon = l*2+3
    #    l += 1
    #    coords.append((lat,lon))
    #print(coords)

    i = 0
    for address in addresses_to_geocode:
        if coords[i] == None:
            geocoded = pd.concat([geocoded,pd.DataFrame([[address, '', '']],columns=headers)])
        else:
            geocoded = pd.concat([geocoded,pd.DataFrame([[address, coords[i][0], coords[i][1]]],columns=headers)])
        i += 1

    existing_geocode.to_csv('cc_geocoded_backup-' + time.strftime("%Y%m%d-%H%M%S") + '.csv',index=False)
    geocoded.to_csv('cc_geocoded.csv',index=False)
    

In [68]:
addresses = ccrr.drop_duplicates(['AddressLine1'])['AddressLine1']
print(addresses.shape)
addresses = addresses.apply(cleanAddressLine1).drop_duplicates()
print(addresses.shape)

(3625,)
(2896,)


In [4]:
existing_geocode = pd.read_csv('cc_geocoded.csv')

In [7]:
existing_geocode[existing_geocode['address'] == address]

2012    34.021223
Name: lat, dtype: float64

In [42]:
# THIS ONE CREATES A GEOJSON

import usaddress
import datetime
import time
import orjson

export_json = dict()
export_json['type'] = 'FeatureCollection'
features = []
addresses = []

flat_json = dict()
flat_json['type'] = 'FeatureCollection'
flat_features = []

existing_geocode = pd.read_csv('cc_geocoded.csv')

for index, row in ccrr.drop_duplicates(['AddressLine1','UnitNumb']).iterrows():
    #TODO: figure out the structure of your JSON
    # I think we want it to be a dictionary of AddressLine1's (and separately we've got a csv that has three columns: addressline1, long, lat)
    # then within that there's a dictionary of features... one of them is a list of units
    # each unit is a dictionary that stores info on a unit, rent history, current rent, beds/baths, etc.
    # OR do we want it stuctured closer to how I want to display it?
    # OR is it more flat so I can use database calls from the JavaScript itself? (i.e., select the 3 beds so we can group by beds when we display?)
    # MAYBE this is creating a JSON that can be displayed on my map and separately I have function to revamp the data for analysis
    # 
    if pd.isna(row['AddressLine1']):
        continue
    

    line1 = row['AddressLine1']
    line2 = row['AddressLine2']
    unit = row['UnitNumb']
    beds = row['NumofBeds']
    baths = row['NumOfBaths']
    built = row['YearBuilt']
    rent = row['RentAmount']
    if pd.isna(row['PastRentReportDate']):
        rentdate = ''
    else:
        rentdate = row['PastRentReportDate'].strftime('%Y/%m/%d')
    encumbered = row['EmcumberedUnit']
    status = row['Unit Status']
    
    address = line1
    if address == '':
        continue
    
    #create geojson for display of addresses only
    if address not in addresses:
        addresses.append(address)
        geo = existing_geocode[existing_geocode['address'] == address]
        feature = dict()
        feature['type'] = 'Feature'

        feature['geometry'] = {
            'type':'Point',
            'coordinates':[geo['lon'].values[0].item(),geo['lat'].values[0].item()]
        }
        
        feature['properties'] = {
            'address':address,
            'units':[[unit, beds, baths, built, rent, rentdate, encumbered, status]],
            'beds':[beds],
            'rent':[rent],
            'rentdate':[rentdate],
            'encumbered':[encumbered == "Yes"],
            'vacant':[status == 'Vacant']
            }
        features.append(feature)
    else:
        for f in features:
            if f['properties']['address'] == address:
                f['properties']['units'].append([unit, beds, baths, built, rent, rentdate, encumbered, status])
                f['properties']['beds'].append(beds)
                f['properties']['rent'].append(rent)
                f['properties']['rentdate'].append(rentdate)
                f['properties']['encumbered'].append(encumbered == "Yes")
                f['properties']['vacant'].append(status == "Vacant")
                break
    
    #create flat_json with a feature for every unit
    flat = dict()
    flat['type'] = 'Feature'

    geo = existing_geocode[existing_geocode['address'] == address]
    flat['geometry'] = {
        'type':'Point',
        'coordinates':[geo['lon'].values[0].item(),geo['lat'].values[0].item()]
    }

    flat['properties'] = {
        'address':address,
        'unit':unit,
        'beds':beds,
        'baths':baths,
        'built':built,
        'rent':rent,
        'rentdate':rentdate,
        'encumbered':encumbered,
        'status':status
    }
    flat_features.append(flat)



for f in features:
    if 'rent' in f['properties'].keys():
        f['properties']['maxRent'] = max(f['properties']['rent'])
        f['properties']['minRent'] = min(f['properties']['rent'])
    if 'beds' in f['properties'].keys():
        f['properties']['maxBeds'] = max(f['properties']['beds'])
        f['properties']['minBeds'] = min(f['properties']['beds'])
    if 'rentdate' in f['properties'].keys():
        f['properties']['maxRentDate'] = max(f['properties']['rentdate'])
        f['properties']['minRentDate'] = min(f['properties']['rentdate'])
    if 'encumbered' in f['properties'].keys():
        f['properties']['any_encumbered'] = any(f['properties']['encumbered'])
        f['properties']['all_encumbered'] = all(f['properties']['encumbered'])
    if 'vacant' in f['properties'].keys():
        f['properties']['any_vacant'] = any(f['properties']['vacant'])
        f['properties']['all_vacant'] = all(f['properties']['vacant'])

#TODO: maybe delete the lists when we're done??? 


export_json['features'] = features
flat_json['features'] = flat_features

with open("ccrr-" + time.strftime("%Y%m%d-%H%M%S") + ".json", "wb") as f:
    f.write(orjson.dumps(export_json))
with open("ccrr-flat-" + time.strftime("%Y%m%d-%H%M%S") + ".json", "wb") as f:
    f.write(orjson.dumps(flat_json))

In [39]:
len(export_json['features'])

2895

In [27]:
import sys
sys.getsizeof(export_json['features'][0])*len(export_json['features'])
len(addresses)

2895

In [25]:
ccrr.drop_duplicates(['AddressLine1']).shape

(2896, 12)

In [29]:
len(export_json['features'])

2895

In [160]:
# OLDD!!! run this to create the json that contains the data to be displayed (this doesn't do GeoJSON... look above for that)

import usaddress
import datetime
import time
import orjson

export_json = dict()

for index, row in ccrr.drop_duplicates(['AddressLine1','UnitNumb']).iterrows():
    #TODO: figure out the structure of your JSON
    # I think we want it to be a dictionary of AddressLine1's (and separately we've got a csv that has three columns: addressline1, long, lat)
    # then within that there's a dictionary of features... one of them is a list of units
    # each unit is a dictionary that stores info on a unit, rent history, current rent, beds/baths, etc.
    # OR do we want it stuctured closer to how I want to display it?
    # OR is it more flat so I can use database calls from the JavaScript itself? (i.e., select the 3 beds so we can group by beds when we display?)
    # MAYBE this is creating a JSON that can be displayed on my map and separately I have function to revamp the data for analysis
    # 
    if pd.isna(row['AddressLine1']):
        continue
    
    line1 = row['AddressLine1']
    line2 = row['AddressLine2']
    unit = row['UnitNumb']
    beds = row['NumofBeds']
    baths = row['NumOfBaths']
    built = row['YearBuilt']
    rent = row['RentAmount']
    if pd.isna(row['PastRentReportDate']):
        rentdate = ''
    else:
        rentdate = row['PastRentReportDate'].strftime('%Y/%m/%d')
    encumbered = row['EmcumberedUnit']
    status = row['Unit Status']
    
    
    #if isinstance(line1,str) and '3959 AND 3961 BENTLEY AVE' in line1:
    #    line1 = '3959-3961 BENTLEY AVE'
    #elif isinstance(line1,str) and '4222 & 4224 JASMINE AVE' in line1:
    #    line1 = '4222-4224 JASMINE AVE'
    #elif isinstance(line1,str) and '4070 LAFAYETTE PL # House B' in line1:
    #    line1 = '4070 LAFAYETTE PL House B'
    #elif isinstance(line1,str) and 'Do not own a rental property in Culver City' in line1:
    #    continue
    #if pd.isna(line2):
    #    line2 = ''
    #tagged_address, address_type= usaddress.tag(str(line1)+', '+str(line2))
    #
    #if ('StreetNamePostType' or 'StreetName' or 'AddressNumber') not in tagged_address.keys():
    #    print(tagged_address)
    #if 'StreetNamePostType' not in tagged_address.keys():
    #    address = ' '.join([tagged_address['AddressNumber'], tagged_address['StreetName']])
    #else:
    #    address = ' '.join([tagged_address['AddressNumber'], tagged_address['StreetName'], tagged_address['StreetNamePostType']])
    
    address = line1
    if address == '':
        continue
    
    if address not in export_json.keys():
        export_json[address] = []
    export_json[address].append([unit, beds, baths, built, rent, rentdate, encumbered, status])

with open("ccrr-" + time.strftime("%Y%m%d-%H%M%S") + ".json", "wb") as f:
    f.write(orjson.dumps(export_json))

In [134]:
tagged_address.keys()

odict_keys(['AddressNumber', 'StreetName', 'StreetNamePostType', 'PlaceName', 'StateName', 'ZipCode'])

In [115]:
import usaddress
import time
import datetime
import orjson 

line1 = '4900  OVERLAND AVE  UNIT 396'
line2 = 'CULVER CITY, CA 90230-4299'
unit = 396
beds = 2
baths = 2
built = 1973
rent = 2725
rentdate = datetime.datetime.strptime('06/18/2022','%m/%d/%Y').date()
encumbered = "No"
status = "Rented"
export_json = dict()

tagged_address, address_type= usaddress.tag(line1+', '+line2)

address = ' '.join([tagged_address['AddressNumber'], tagged_address['StreetName'], tagged_address['StreetNamePostType']])
export_json[address] = [unit, beds, baths, built, rent, rentdate, encumbered, status]

with open("ccrr-" + time.strftime("%Y%m%d-%H%M%S") + ".json", "wb") as f:
    f.write(orjson.dumps(export_json))


In [114]:
time.strftime("%Y%m%d-%H%M%S")

'20230129-201338'

In [94]:
outliers = pd.DataFrame(columns=list(ccrr.columns))
for index, row in ccrr.drop_duplicates(['AddressLine1','UnitNumb'])[['AddressLine1','UnitNumb']].iterrows():
    address = row['AddressLine1']
    unit_number = row['UnitNumb']
    # todo: print address and unit_number, wait 15minutes and see what's crashing this next line
    subset = ccrr[(ccrr['AddressLine1'].str.contains(address)) & ((ccrr['UnitNumb']==unit_number)|(ccrr['UnitNumb'].str.contains(str(unit_number)))) & (ccrr['PastRentReportDate'].notna())]
    subset = subset.sort_values(by='PastRentReportDate',ascending=False).reset_index()
    a = subset
    if subset.empty:
        continue
    if (subset['PastAmount'].iloc[0] != subset['RentAmount'].iloc[0]):
        outliers = pd.concat([outliers,subset.loc[[0]]])
print(outliers)

<ipython-input-94-541f0dd35c04>:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  subset = ccrr[(ccrr['AddressLine1'].str.contains(address)) & ((ccrr['UnitNumb']==unit_number)|(ccrr['UnitNumb'].str.contains(str(unit_number)))) & (ccrr['PastRentReportDate'].notna())]


error: nothing to repeat at position 0

In [95]:
outliers

,AddressLine1,AddressLine2,UnitNumb,NumofBeds,NumOfBaths,YearBuilt,RentAmount,PastRentReportDate,PastAmount,EmcumberedUnit,Unit Status


In [2]:
from mapbox import Geocoder

geocoder = Geocoder(access_token='pk.eyJ1IjoiaHVnaGZpdHpnZXJhbGQiLCJhIjoiY2xkZGhjaG9wMDNqdTNvdDZ5bG80OXZ3YSJ9.6njmVnJyl0zAtMnM9d8duQ')
response = geocoder.forward('4252 Coolidge Ave, Los Angeles, CA 90066')

<Response [200]>


In [6]:
response.json()['features'][0]

{'id': 'address.7082831410212634',
 'type': 'Feature',
 'place_type': ['address'],
 'relevance': 1,
 'properties': {'accuracy': 'rooftop'},
 'text': 'Coolidge Avenue',
 'place_name': '4252 Coolidge Avenue, Los Angeles, California 90066, United States',
 'center': [-118.4171, 34.00101],
 'geometry': {'type': 'Point', 'coordinates': [-118.4171, 34.00101]},
 'address': '4252',
 'context': [{'id': 'neighborhood.394022124', 'text': 'McLaughlin'},
  {'id': 'postcode.300560108', 'text': '90066'},
  {'id': 'place.192407788', 'wikidata': 'Q65', 'text': 'Los Angeles'},
  {'id': 'district.14051052',
   'wikidata': 'Q104994',
   'text': 'Los Angeles County'},
  {'id': 'region.419052',
   'short_code': 'US-CA',
   'wikidata': 'Q99',
   'text': 'California'},
  {'id': 'country.8940',
   'short_code': 'us',
   'wikidata': 'Q30',
   'text': 'United States'}]}

In [7]:
address = '6605  GREEN VALLEY CIR  UNIT 113	CULVER CITY, CA 90230-8083'
response = geocoder.forward(address)
response.json()['features'][0]

{'id': 'address.5063949685794286',
 'type': 'Feature',
 'place_type': ['address'],
 'relevance': 1,
 'properties': {'accuracy': 'rooftop'},
 'text': 'Green Valley Circle',
 'place_name': '6605 Green Valley Circle, Culver City, California 90230, United States',
 'center': [-118.385796, 33.97883],
 'geometry': {'type': 'Point', 'coordinates': [-118.385796, 33.97883]},
 'address': '6605',
 'context': [{'id': 'neighborhood.229092588', 'text': 'Fox Hills'},
  {'id': 'postcode.300904172', 'text': '90230'},
  {'id': 'place.77531372', 'wikidata': 'Q493378', 'text': 'Culver City'},
  {'id': 'district.14051052',
   'wikidata': 'Q104994',
   'text': 'Los Angeles County'},
  {'id': 'region.419052',
   'short_code': 'US-CA',
   'wikidata': 'Q99',
   'text': 'California'},
  {'id': 'country.8940',
   'short_code': 'us',
   'wikidata': 'Q30',
   'text': 'United States'}]}

In [9]:
import usaddress
usaddress.parse(address)

[('6605', 'AddressNumber'),
 ('GREEN', 'StreetName'),
 ('VALLEY', 'StreetName'),
 ('CIR', 'StreetNamePostType'),
 ('UNIT', 'OccupancyType'),
 ('113', 'OccupancyIdentifier'),
 ('CULVER', 'PlaceName'),
 ('CITY,', 'PlaceName'),
 ('CA', 'StateName'),
 ('90230-8083', 'ZipCode')]

In [10]:
address = '9770 CULVER BLVD	CULVER CITY, CA 90232-2703'
response = geocoder.forward(address)
response.json()['features'][0]

{'id': 'address.766468067844202',
 'type': 'Feature',
 'place_type': ['address'],
 'relevance': 1,
 'properties': {'accuracy': 'rooftop'},
 'text': 'Culver Boulevard',
 'place_name': '9770 Culver Boulevard, Culver City, California 90232, United States',
 'center': [-118.39559, 34.02163],
 'geometry': {'type': 'Point', 'coordinates': [-118.39559, 34.02163]},
 'address': '9770',
 'context': [{'id': 'neighborhood.375500012', 'text': 'Lucerne'},
  {'id': 'postcode.300920556', 'text': '90232'},
  {'id': 'place.77531372', 'wikidata': 'Q493378', 'text': 'Culver City'},
  {'id': 'district.14051052',
   'wikidata': 'Q104994',
   'text': 'Los Angeles County'},
  {'id': 'region.419052',
   'short_code': 'US-CA',
   'wikidata': 'Q99',
   'text': 'California'},
  {'id': 'country.8940',
   'short_code': 'us',
   'wikidata': 'Q30',
   'text': 'United States'}]}

In [11]:
address = '4396 KEYSTONE AVE	CULVER CITY, CA 90232-3437'
response = geocoder.forward(address)
response.json()['features'][0]

{'id': 'address.1144368606704674',
 'type': 'Feature',
 'place_type': ['address'],
 'relevance': 1,
 'properties': {'accuracy': 'rooftop'},
 'text': 'Keystone Avenue',
 'place_name': '4396 Keystone Avenue, Culver City, California 90232, United States',
 'center': [-118.39549, 34.009255],
 'geometry': {'type': 'Point', 'coordinates': [-118.39549, 34.009255]},
 'address': '4396',
 'context': [{'id': 'neighborhood.91180268', 'text': 'Carlson'},
  {'id': 'postcode.300920556', 'text': '90232'},
  {'id': 'place.77531372', 'wikidata': 'Q493378', 'text': 'Culver City'},
  {'id': 'district.14051052',
   'wikidata': 'Q104994',
   'text': 'Los Angeles County'},
  {'id': 'region.419052',
   'short_code': 'US-CA',
   'wikidata': 'Q99',
   'text': 'California'},
  {'id': 'country.8940',
   'short_code': 'us',
   'wikidata': 'Q30',
   'text': 'United States'}]}